# Xsarsea example

The Normalized Radar Cross Section (sigma0) as computed from Level-1 SAR data can be detrended in the case of ocean scenes. 

The goal is to remove the averaged trend (decreasing) of the NRCS with (increasing) incidence angle observed for acquisitions over ocean. 

The detrend maximizes the contrasts in the image due to geophysical phenomena and improves the visualization experience of ocean scenes. sigma0_detrend is also termed **image roughness** or **nice display**.

In [ ]:
import xsar
import xsarsea
import os

In [ ]:
# use holoviews for plots

import holoviews as hv
hv.extension('bokeh')
from holoviews.operation.datashader import datashade,rasterize
import datashader as dh

## read the dataset with xsar

In [ ]:
# get test file. You can replace with an path to other SAFE
filename = xsar.get_test_file('S1A_IW_GRDH_1SDV_20170907T103020_20170907T103045_018268_01EB76_Z010.SAFE')
filename

In [ ]:
# open the dataset with xsar
sar_ds = xsar.open_dataset(filename, resolution={'atrack':10,'xtrack':10})
sar_ds[['sigma0','incidence']]


## Sigma0 detrending

Sigma0 detrending is done by  [xsarsea.sigma0_detrend](../basic_api.rst#xsarsea.sigma0_detrend) function

As the resulting xarray dataset have the same coordinates as the original sigma0, we can add a `sigma0_detrend` variable to the dataset.

In [ ]:
sar_ds['sigma0_detrend'] = xsarsea.sigma0_detrend(sar_ds.sigma0, sar_ds.incidence)

In [ ]:
rasterize(hv.Image(sar_ds.sigma0.sel(pol='VV')).opts(cmap='gray',colorbar=True,tools=['hover'],title="original sigma0")) + rasterize(hv.Image(sar_ds.sigma0_detrend.sel(pol='VV')).opts(cmap='gray',colorbar=True,tools=['hover'],title="detrended sigma0"))

## Wind streaks direction

(⚠️work in progress⚠️ )

More science work is needed to have better results

In [ ]:
import xsarsea.streaks
import xarray as xr

Wind streaks detection is done with [xsarsea.streaks.streaks_direction](../basic_api.rst#xsarsea.streaks.streaks_direction)

The resulting datarray doesn't have the same shape as `sigma0_detrend`, so it can't be included in the original dataset (or an interpolation is needed)

In [ ]:
streaks_dir = xsarsea.streaks.streaks_direction(
    sar_ds['sigma0_detrend'])
streaks_dir

In [ ]:
rasterize(hv.Image(sar_ds['sigma0_detrend'].sel(pol='VV'))).opts(cmap='gray',frame_width=400, frame_height=400) * hv.VectorField( xr.ufuncs.deg2rad(streaks_dir.sel(pol='VV'))).opts(arrow_heads=False) 